In [1]:
# %pip install statsmodels
# %pip install mlxtend
# %pip install xgboost

In [2]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
import itertools
import random
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xgboost as xgb
from sklearn.model_selection import train_test_split
import random
import pandas as pd

C:\Users\austinsh\AppData\Local\Temp\ipykernel_52976\526159370.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

df_All = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C10\3 Feature Selection\filtered-out_5-9_corr.csv')
print(df_All.head())

df_TagDesc = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\II Data\1 Collection\CSV\Not for Processing\TagDesc.csv')


                  Date  425_pct_Al     Al2O3  M_Value  C4_pct_Eth  C4_pct_H2O  \
0  2012-05-17 09:00:00    6.319560  11.41670  3.50773    2.554580     22.0531   
1  2012-05-17 10:00:00    6.319915  11.40835  3.50797    2.555935     22.0557   
2  2012-05-17 12:00:00    6.320970  11.39165  3.50869    2.560005     22.0636   
3  2012-05-18 08:00:00    6.328690  11.39165  3.51401    2.540370     22.0604   
4  2012-05-18 09:00:00    6.336060  11.40500  3.51910    2.514430     22.0484   

   HydWtr_pct_Ammonia  C4_pct_Hex  HydWtr_Na2O  Decanol  ...   LC55557  \
0            0.867508    0.670721     3.354160  1.03026  ...  66.31395   
1            0.864729    0.671059     3.306475  1.03026  ...  66.31395   
2            0.856391    0.672071     3.163420  1.06326  ...  66.31395   
3            0.811652    0.674174     2.238130  1.51850  ...  65.92560   
4            0.773793    0.675408     1.391475  1.51850  ...  65.92560   

    LC55568    LC90366   LC90368   PI55004   PI55020   TC55552   TC5

In [4]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [5]:
# df_All = df_All[df_All['Date'] > '2022-06-15 00:00:00']

In [6]:
# selected_columns = ['Butanol', 
#                     'FC55569',
#                     'DI55152',
#                     '425 %Al',
#                     'TC55552',
#                     'FC55003',
#                     'LC55555',
#                     'FFC55553',
#                     'FFC55555',
#                     'TC55555',
#                     'TI55021',
#                     'PI55004',
#                     'FC55552'
#                     ]
# existing_columns = [col for col in selected_columns if col in df_All.columns]
# df_All = df_All[existing_columns]


In [7]:
# # List of columns to exclude to run XGboost feature selection
exclude_columns = [
                'TI40050',
                'TC55555',
                'LC55568',
                'LC55557',
                'Al2O3',
                'LC55553',
                'M_Value',
                'FFC55553',
                'DI55580',
                'Al2O3',
                'C4_pct_Hex',
                'LC52572',

        # 'Octanol', 'Hexanol',
#        'Ethanol', 'Decanol',
       
#        'TI52014', 'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021', 'TI55023',
#        'TC52015', 'FC52018', 'II52554', 'TI40050', 'VI52558B'

#        # 'FC55102', 'FC55152', 'LC55557', 'LC55568', 'TC55555',

#        # '425 SAO Al', 'FFC55553', 'LC52572', 'LC90366',

#        # 'FC42428', 'LC55553',

#        # 'FC55009'
                   ]

# # Create a new DataFrame without the excluded columnsd
df_All = df_All.drop(columns=exclude_columns)

In [8]:
df_All.columns

Index(['Date', '425_pct_Al', 'C4_pct_Eth', 'C4_pct_H2O', 'HydWtr_pct_Ammonia',
       'HydWtr_Na2O', 'Decanol', 'DI55102', 'DI55152', 'FC55003', 'FC55009',
       'FC55552', 'FC55569', 'FC55576', 'FFC55555', 'LC90366', 'LC90368',
       'PI55004', 'PI55020', 'TC55552', 'TI55021'],
      dtype='object')

In [9]:
# # Splitting into train and test
# X = df_All.drop('Butanol', axis=1)  # Assuming 'target' is your target column
# y = df_All['Butanol']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
def iterate_feature_rotations(df_all, target_column, test_size=0.2, random_state=42, num_random_iterations=30):
    results = []
    columns = [col for col in df_all.columns if col != target_column and col != 'Date']
    random.seed(random_state)  # for reproducibility

    for feature in columns:
        for _ in range(num_random_iterations):
            # Randomly order the remaining features
            remaining_features = [f for f in columns if f != feature]
            random.shuffle(remaining_features)

            # Create a new ordered list of features
            ordered_features = [feature] + remaining_features

            reordered_df = df_all[ordered_features + [target_column]]

            # Splitting into train and test for each permutation
            X = reordered_df.drop(target_column, axis=1)
            y = reordered_df[target_column]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

            # Create and fit the XGBoost model
            model = xgb.XGBRegressor(objective='reg:squarederror')
            model.fit(X_train, y_train)

            # Extract feature importances
            feature_importances = model.get_booster().get_score(importance_type="weight")

            # Store the result with the permutation order and feature importances
            results.append((ordered_features, feature_importances))

    return results


results = iterate_feature_rotations(df_All, 'Decanol')

# Convert results to a DataFrame
flattened_results = []
for ordered_features, importances in results:
    for feature, importance in importances.items():
        flattened_results.append({
            'Feature Rotation': ordered_features,
            'Feature': feature,
            'Importance': importance
        })

results_df = pd.DataFrame(flattened_results)


In [12]:
print(results_df)

                                        Feature Rotation     Feature  \
0      [425_pct_Al, LC90368, LC90366, DI55102, FC5556...  425_pct_Al   
1      [425_pct_Al, LC90368, LC90366, DI55102, FC5556...     LC90368   
2      [425_pct_Al, LC90368, LC90366, DI55102, FC5556...     LC90366   
3      [425_pct_Al, LC90368, LC90366, DI55102, FC5556...     DI55102   
4      [425_pct_Al, LC90368, LC90366, DI55102, FC5556...     FC55569   
...                                                  ...         ...   
10825  [TI55021, FC55552, FC55003, LC90366, PI55020, ...     PI55004   
10826  [TI55021, FC55552, FC55003, LC90366, PI55020, ...  C4_pct_Eth   
10827  [TI55021, FC55552, FC55003, LC90366, PI55020, ...     DI55152   
10828  [TI55021, FC55552, FC55003, LC90366, PI55020, ...     DI55102   
10829  [TI55021, FC55552, FC55003, LC90366, PI55020, ...  425_pct_Al   

       Importance  
0           493.0  
1           394.0  
2           349.0  
3           343.0  
4           273.0  
...           .

In [13]:
# Group by 'Feature' and calculate the average importance
average_importances = results_df.groupby('Feature')['Importance'].mean()

# Convert the Series to a DataFrame
average_importances_df = average_importances.reset_index()

# Rename the columns for clarity
average_importances_df.columns = ['Feature', 'Average Importance']

# Sort the DataFrame by 'Average Importance' in descending order
average_importances_df = average_importances_df.sort_values(by='Average Importance', ascending=False)

# Display or save the DataFrame
# print(average_importances_df)
# Or save it to a CSV file
# average_importances_df.to_csv('average_feature_importances.csv', index=False)


In [14]:
# Merge the average_importances_df with df_TagDesc
# Assuming 'ID' in df_TagDesc corresponds to 'Feature' in average_importances_df
merged_df = average_importances_df.merge(df_TagDesc, left_on='Feature', right_on='ID', how='left')

# Select only the required columns
final_df = merged_df[['Feature', 'Average Importance', 'DESCRIPTION']]

# Rename the 'DESCRIPTION' column to 'Description'
final_df.rename(columns={'DESCRIPTION': 'Description'}, inplace=True)

# Display or save the DataFrame
print(final_df)
# Or save it to a CSV file
# final_df.to_csv('average_feature_importances_with_descriptions.csv', index=False)


               Feature  Average Importance               Description
0           C4_pct_H2O          337.747368                       NaN
1          HydWtr_Na2O          334.901754                       NaN
2           C4_pct_Eth          333.024561                       NaN
3              LC90368          328.135088      FB-658 ALKOXIDE TANK
4              DI55102          317.489474          HYDROL RX OUTLET
5              TC55552          316.207018  DC-551 ALKOX  FD PREHEAT
6              FC55552          305.059649  ALK FD TO HYDR RX DC-551
7              LC90366          303.568421        FB-657 SAO STORAGE
8   HydWtr_pct_Ammonia          298.978947                       NaN
9              DI55152          294.936842    2ND STG SEPR TO FA-560
10             FC55009          286.077193   STM TO PA-551  RING HTR
11          425_pct_Al          283.631579                       NaN
12             PI55020          268.928070       DA-554 TOP PRESSURE
13             FC55569          25

C:\Users\austinsh\AppData\Local\Temp\ipykernel_52976\3091506829.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'DESCRIPTION': 'Description'}, inplace=True)


In [15]:
from datetime import datetime

# Get the current date and time
current_date_time = datetime.now()

# Print the current date and time
print(current_date_time)


2024-03-21 18:56:21.278696
